## QCoDeS Training for Developers

Welcome to the training about QCoDeS targeted for developers. After this training, developers should have more knowledge about QCoDeS which will bring them confidence with contributing to it.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

### Approach

- Don't bring too much relevant background, only small absolutely necessary portions
- Provide refresh of user-side view on QCoDeS
- Describe features/concepts from ideas, but get close to code very quickly

### Execution plan for presenter

- Before:
  - Have qcodes code at hand in your IDE https://github.com/qcodes/qcodes
  - Run the training Jupyter notebook, and have it at hand https://github.com/astafan8/Qcodes/blob/training-for-devs/docs/examples/QCoDeS%20Training%20for%20Developers.ipynb
  - Familiarize yourself with the following info from the docs:
    - Front page https://qcodes.github.io/Qcodes/index.html
    - 15 minutes of QCoDeS https://qcodes.github.io/Qcodes/examples/15_minutes_to_QCoDeS.html
    - `Parameter`s
      - overview1 https://qcodes.github.io/Qcodes/examples/Parameters/Parameters.html
      - overview2 https://qcodes.github.io/Qcodes/examples/writing_drivers/Creating-Instrument-Drivers.html#What%E2%80%99s-a-Parameter? - only this section
      - API docs https://qcodes.github.io/Qcodes/api/parameters/parameter.html#module-qcodes.instrument.parameter
      - `ParameterWithSetpoints` https://qcodes.github.io/Qcodes/examples/Parameters/Simple-Example-of-ParameterWithSetpoints.html
    - Performing measurements https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html
    - dataset notebooks
      - the class https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html
      - data access https://qcodes.github.io/Qcodes/examples/DataSet/Accessing-data-in-DataSet.html
- During:
  - Go through notebook in 3 parts, have 5 min breaks, 5 min reserve beginning, 10 min reserve at the end
  - Have a time keeper
  - Let ask questions at specific moments but very frequently to feel interactive
- After:
  - Port notebook to qcodes docs ASAP
  - Sessions on some parts if requested

### Imports

In [1]:
import qcodes

from typing import Optional, Sequence, Dict
from pprint import pprint

import numpy

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-miasta\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-miasta\.qcodes\logs\200409-16380-qcodes.log


## QCoDeS? ah? [5min]

What is it? What are it's goals, and users? And why?

### "The It"

**Q.**uantum **Co**penhagen **De**lft **S**ydney:

- data acquisition framework
- modular
- python-based
- open-source

### Goal

**Common** framework for physics experiments:

- users need to write only their-own-experiment-specific code
- physics experiments can take advantage of modern software and best practices
- code can and should be contributed back to the framework
- the process of moving between teams or labs, and of setting up a new experiment is streamlined
- new students don’t need to spend a long time learning software in order to participate in experiments

### Users

- people doing measurements:
  - academics: e.g. scientists and students
  - corporates: e.g. experimentalists, engineers, hardware manufacturers
- worldwide

### Driven by

- Microsoft, largely
- worldwide comminity, growing

### Development principles

- Robust code - as opposed to what users are used to from other measurement frameworks
- Respect backwards compatibility - don't break user's code that "works"!
- Modularity - extend-existing, not go-off-and-write-your-own
- User facing API - list in docs, devs can use everything as appropriate
- Carefully scoping problems

## I wanna MEASURE [30min]

How do I measure things? How are measurements enabled via QCoDeS? Station, instruments, parameters?

- Quickly about general stuff
- Deeper into parameters, snapshot

### Station

``Station`` is just a bucket for components such as instruments, parameters, etc. Speaking more strictly, those components should be snapshot-able (subclasses of ``Metadatable``), but more about this in the section about the snapshot.

``Station`` can be created with python code and/or using YAML configuration file. [Read on Station, its interface, YAML config, and more here](http://qcodes.github.io/Qcodes/examples/Station.html).

In [2]:
from qcodes.station import Station

#### Default station

Notice the ``default=True`` argument of the ``Station`` - it controls wether to store the initialized ``Station`` object into an attribute.

Using ``Station.default`` as a fallback in other objects in QCoDeS is very common, as it is extremely uncommon that more than one ``Station`` is initiated in one python session.

In [3]:
station = Station()  # default=True

assert Station.default is station

station2 = Station(default=False)

assert station2 is not Station.default

### Instruments

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

	• Instruments - just a bucket of parameters and hardware interaction
		○ Base vs not Base
		○ write/ask methods
		○ Visa vs DLL vs ???
		○ Registry of instrument instances - how does it work and how to see which instances exists etc.
		○ Submodules, Channels, channel lists
		○ Logging - logger instance on each instance
		○ DelegateAttributes
		○ Writing drivers notebook - skip


### Parameters

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

	• Parameters
		○ What they are - two purposes
		○ Name/label/unit/docstring - metadata
		○ Set/get
			§ _raw
			§ command
		○ instrument's add_parameter         
		○ Set_to
		○ Cache and get_latest
		○ Value/raw_value
			§ Parsers
			§ Val_mapping
			§ Validators
		○ Snapshot - it's nontriviallity and flags
		○ More specific parameters:
			§ Delegate parameter
			§ Array + setpoints
			§ Multiparameter
			§ Group parameter
			§ How to subclass in general

### Snapshot

Snapshot is a "photograph" of a state of a measurement setup or its piece, be it ``Station``, ``Instrument``, ``Parameter``, or other components.

Snapshot is **not** maintained in memory. Instead, each snapshot-able component can always return/produce its snapshot.

#### Subclass ``Metadatable``

Snapshot-able objects should inherit from ``qcodes.utils.metadata.Metadatable`` and should override ``snapshot_base`` method to return a dictionary representing the state of the object. Then, calling ``snapshot()`` method will return the snapshot of the object.

In [4]:
from qcodes.utils.metadata import Metadatable


class TeamsStatus(Metadatable):
    def __init__(self, name: str, status: str, metadata=None):
        super().__init__(metadata=metadata)
        self.name = name
        self.status = status

    def snapshot_base(
            self, update: bool = False,
            params_to_skip_update: Optional[Sequence[str]] = None) -> Dict:
        return {'status': self.status}


teams_status = TeamsStatus('Fridge25 status', 'busy')

pprint(teams_status.snapshot())

{'status': 'busy'}


####  ``metadata`` argument

Note that ``Metadatable`` also implements ``metadata`` keyword argument - ``metadata`` is expected to be a dictionary, and it will be included in the snapshot automatically (if it is not ``{}``).

In [5]:
favorites = Metadatable(metadata={'number': 6 + 9j})
favorites.load_metadata({'fruit': 'manadrine'})  # ``dict.update`` behavior

In [6]:
pprint(favorites.snapshot())

{'metadata': {'fruit': 'manadrine', 'number': (6+9j)}}


#### Add  ``Metadatable`` to station

``Metadatable`` objects can be added to the ``Station``, other objects - should not (otherwise ``snapshot()`` will raise).

In [7]:
station = Station(teams_status)
station.add_component(favorites)

'component1'

In [8]:
pprint(station.snapshot())

{'components': {'Fridge25 status': {'status': 'busy'},
                'component1': {'metadata': {'fruit': 'manadrine',
                                            'number': (6+9j)}}},
 'config': None,
 'default_measurement': [],
 'instruments': {},
 'parameters': {}}


#### Create ``JSON`` snapshot

Although snapshot is a python dictionary, it is always meant to be converted to JSON and stored next to the measured data. QCoDeS maintains it's own JSON converter class, ``qcodes.utils.helpers.NumpyJSONEncoder``, to support conversion of objects that are frequently met in measurement world, such as ``numpy`` arrays, complex numbers, pickle-ables.

In [9]:
from qcodes.utils.helpers import NumpyJSONEncoder
import json

json_snapshot = json.dumps(
    station.snapshot(), cls=NumpyJSONEncoder,
    indent=2, sort_keys=True  # for pretty-printing
)

In [10]:
print(json_snapshot)

{
  "components": {
    "Fridge25 status": {
      "status": "busy"
    },
    "component1": {
      "metadata": {
        "fruit": "manadrine",
        "number": {
          "__dtype__": "complex",
          "im": 9.0,
          "re": 6.0
        }
      }
    }
  },
  "config": null,
  "default_measurement": [],
  "instruments": {},
  "parameters": {}
}


#### ``update`` arguments

Note that ``snapshot`` has an ``update`` kwarg, and ``snapshot_base`` has ``update`` and ``params_to_skip_update`` kwargs - those allow to optimize querying the state of a ``Metadatable`` object.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

## BREAK [5min]

## I wanna STORE [30min]

How and where do I store what I measured? And access back? Database, experiments, datasets, parameters?

- Quickly DB structure, paramtypes, data saving approach
- mention DataSet is too close to sqlite
- dig deeper into how Measurement/DataSet are designed

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)


	• How people measure - interactively!
		○ Why loop and not loop anymore
		○ Measurement as general thing and create-your-own-measurement-function
		○ Dond, pysweep, etc.
	• Measurement -> DataSet
		○ ASSUME THAT 'how to use them' IS KNOWN
		○ Name, experiment, station
		○ Parameters
			§ Register_* - as interface between qcodes parameters and dataset parameters
			§ Rundescription/interdeps/paramspecbase
			§ paramtype
		○ Before and after actions :)
		○ .run() -> Runner -> DataSaver - what is their role?
			§ Runner
			§ DataSaver
				□ Add_results
					® Validation that happens
				□ Dataset/run_id
		○ Sqlite data saving optimizations
			§ Paramtype "array"
			§ wrtie_period
			§ write_in_background
		○ Subscribers - bonus topic
	• Database stuff
		○ Runs table as log of performed measurements
		○ Experiment as a bucket of datasets but not used as such really
			§ Ideas for "active experiment, etc"
		○ Data storage
			§ As a message queue/log - needs reconstruction
			§ in the same DB
		○ Sqlite database, schema, versions
			§ Connect - users might forget to close, etc
		○ Sqlite module overview
	• DataSet walkthrough
		○ Identity, also timestamps and state
		○ parameters
		○ Getting data
		○ Metadata feature


## BREAK [5min]

## I want MORE [5min]

What else does QCoDeS provide out of the box?

- q/a, other qcodes topics for self-exploration

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

- contrib-drivers
- Config
- DelegateAttributes
- Plotting support
- Instrument's functions
- Deprecation
- Linking datasets
- Extracting runs/datasets from db to another
- Monitor
- Interactive widget
- Legacy, loop-entangled, features

## Presenter-only section

Some info, notes, and scratch-space for presenter.

qcodes logo means that the section(s) under it's level is(are) not ready.